In [2]:
spark.version

'3.1.3'

In [3]:
import time
import pandas as pd
from itertools import islice

In [4]:
from google.cloud import storage

In [5]:
# List all files in given COS directory
def list_blobs(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    for blob in blobs:
        print(blob.name + '\t' + str(blob.size))

In [6]:
# List all files in given COS directory
def list_blobs_pd(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    blob_name = []
    blob_size = []
    
    for blob in blobs:
        blob_name.append(blob.name)
        blob_size.append(blob.size)

    blobs_df = pd.DataFrame(list(zip(blob_name, blob_size)), columns=['Name','Size'])

    blobs_df = blobs_df.style.format({"Size": "{:,.0f}"}) 
    
    return blobs_df

In [7]:
# Delete folder from COS bucket
def delete_folder(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    for blob in blobs:
        blob.delete()

In [8]:
import os
import subprocess
import shutil
import pandas as pd
# import sh
from pyspark.sql.functions import *
#from pyspark.sql import functions as F
from pyspark.sql.types import *

In [9]:
#BDP Assignment 8

In [14]:
!hadoop fs -ls "gs://msca-bdp-amazon/amazon_reviews/"

Nov 07, 2023 8:09:47 AM com.google.cloud.hadoop.fs.gcs.GhfsStorageStatistics updateMinMaxStats
INFO: Detected potential high latency for operation op_get_file_status. latencyMs=188; previousMaxLatencyMs=0; operationCount=1; context=gs://msca-bdp-amazon/amazon_reviews
Nov 07, 2023 8:09:47 AM com.google.cloud.hadoop.fs.gcs.GhfsStorageStatistics updateMinMaxStats
INFO: Detected potential high latency for operation op_glob_status. latencyMs=253; previousMaxLatencyMs=0; operationCount=1; context=path=gs://msca-bdp-amazon/amazon_reviews; pattern=com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystemBase$$Lambda$8/330739404@3dd1dc90
Found 46 items
-rwx------   3 root root 1971061630 2020-10-28 20:26 gs://msca-bdp-amazon/amazon_reviews/amazon_reviews_us_Apparel_v1_00.tsv
-rwx------   3 root root 1350294084 2020-10-28 20:26 gs://msca-bdp-amazon/amazon_reviews/amazon_reviews_us_Automotive_v1_00.tsv
-rwx------   3 root root  872274720 2020-10-28 20:26 gs://msca-bdp-amazon/amazon_reviews/amazon_rev

In [10]:
bucket = "msca-bdp-amazon"
folder = "amazon_reviews/"

In [18]:
#GCS bucket "gs://msca-bdp-amazon/amazon_reviews/"
files = spark.read.option("delimiter", "\t").option("header", "true").csv(f"gs://{bucket}/{folder}*.tsv")

In [12]:
files.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [18]:
type(files)

pyspark.sql.dataframe.DataFrame

In [19]:
category_reviews = files.groupBy("product_category").count()

In [20]:
most_reviews_category = category_reviews.orderBy(col("count").desc()).first()

In [21]:
#Which product category gets most reviews
print("Product category with the most reviews:")
print(f"Category: {most_reviews_category['product_category']}")
print(f"Number of Reviews: {most_reviews_category['count']}")

Product category with the most reviews:
Category: Books
Number of Reviews: 19531194


In [ ]:
#Product category with the most reviews:
#Category: Books
#Number of Reviews: 19531194

In [22]:
#only select the relevant columns in our analysis
files1 = files.select("customer_id", "review_id", "product_category", "star_rating", "helpful_votes", "total_votes", "review_headline", "review_body", "review_date" )

In [23]:
#removing/droping records with NULL values as it will not contribute/help with our analysis
files2 = files1.na.drop()

In [ ]:
#Above 2 steps reduces the amount/size (columns & rows) of the dataset we are working with to help with the computational effectiveness of our jobs.

In [24]:
#Which product category gets highest / lowest review star ratings
files2.createOrReplaceTempView("amazon_products")

In [25]:
product_category_with_highest_avg_star_rating = spark.sql("""
    SELECT product_category, AVG(star_rating) AS avg_rating
    FROM amazon_products
    GROUP BY product_category
    ORDER BY avg_rating DESC
    LIMIT 1
""")

In [26]:
#product category with highest review star ratings
product_category_with_highest_avg_star_rating.show()

+----------------+-----------------+
|product_category|       avg_rating|
+----------------+-----------------+
|       Gift Card|4.731352294070298|
+----------------+-----------------+



In [ ]:
#Gift Card has the highest average rating with 4.731363105858364.

In [27]:
product_category_with_lowest_avg_star_rating = spark.sql("""
    SELECT * FROM (
    SELECT product_category, AVG(star_rating) AS avg_rating
    FROM amazon_products
    GROUP BY product_category
    ) AS T1
    WHERE avg_rating is not NULL
    ORDER BY avg_rating ASC
    LIMIT 1
""")

In [28]:
#product category with lowest review star ratings
product_category_with_lowest_avg_star_rating.show()

+----------------+------------------+
|product_category|        avg_rating|
+----------------+------------------+
|Digital_Software|3.5393869333934185|
+----------------+------------------+



In [ ]:
#Digital Software has the lowest average rating with 3.5393303553935973.

In [ ]:
#Do you see seasonality in reviews? 
#Is every product category getting consistent number of reviews throughout the year?
#Any product categories that are getting more reviews during certain times of the year?  If yes, which product categories and what times of the year?

In [ ]:
seasonal_review_count = spark.sql("""
    SELECT seasonality, COUNT(*) FROM
    (
    SELECT
    CASE
        WHEN MONTH(review_date) IN (12, 1, 2) THEN 'Winter'
        WHEN MONTH(review_date) IN (3, 4, 5) THEN 'Spring'
        WHEN MONTH(review_date) IN (6, 7, 8) THEN 'Summer'
        WHEN MONTH(review_date) IN (9, 10, 11) THEN 'Fall'
    END AS seasonality
    FROM amazon_products) AS T1
    GROUP BY seasonality
    order by 2 desc
    ;
""")

In [ ]:
seasonal_review_count.show()

+-----------+--------+
|seasonality|count(1)|
+-----------+--------+
|     Summer|41861561|
|     Winter|41653457|
|     Spring|38454396|
|       Fall|28978989|
|       null|   13875|
+-----------+--------+



In [37]:
product_category_seasonal_review_count = spark.sql("""
    SELECT *, COUNT(*) FROM
    (
    SELECT product_category,
    CASE
        WHEN MONTH(review_date) IN (12, 1, 2) THEN 'Winter'
        WHEN MONTH(review_date) IN (3, 4, 5) THEN 'Spring'
        WHEN MONTH(review_date) IN (6, 7, 8) THEN 'Summer'
        WHEN MONTH(review_date) IN (9, 10, 11) THEN 'Fall'
    END AS seasonality
    FROM amazon_products) AS T1
    GROUP BY product_category, seasonality
    order by 3 desc
    ;
""")

In [ ]:
product_category_seasonal_review_count.show()

+--------------------+-----------+--------+
|    product_category|seasonality|count(1)|
+--------------------+-----------+--------+
|               Books|     Winter| 5249304|
|               Books|     Summer| 5130555|
|               Books|     Spring| 4917451|
|Digital_Ebook_Pur...|     Summer| 4916200|
|Digital_Ebook_Pur...|     Spring| 4838158|
|Digital_Ebook_Pur...|     Winter| 4374698|
|               Books|       Fall| 4230898|
|Digital_Ebook_Pur...|       Fall| 3491205|
|            Wireless|     Summer| 2596900|
|            Wireless|     Winter| 2448837|
|            Wireless|     Spring| 2252281|
|                  PC|     Winter| 2112010|
|                Home|     Winter| 1844690|
|                Toys|     Winter| 1825568|
|                  PC|     Summer| 1798630|
|                Home|     Summer| 1727559|
|                  PC|     Spring| 1705586|
|            Wireless|       Fall| 1703573|
|             Apparel|     Winter| 1668663|
|             Apparel|     Summe

In [ ]:
#Overall, the summer season has the highest number of reviews, totaling 41,861,561. Winter closely follows as the second season with 41,653,457 reviews. The fall season lags behind with the lowest number of reviews, totaling 28,978,989.
#It appears that there is indeed a seasonality in reviews. Winter seems to be the season where most products receive the most reviews (such as Books, PC, Home, Toys, Apparel). However, for some products, Summer are also sometimes the season that receives the most reviews (such as Digital Ebooks and Wireless). On the other hand, there is a commonality in the majority of product categories where Fall would be the season with the least amount of reviews!

In [ ]:
#Which reviews are getting the most helpful votes / total votes?
#Any correlation between the length of review headline / review body (in number words) and the "helpfulness" of the review?

In [15]:
most_total_votes = spark.sql("""
    SELECT review_id, sum(total_votes)
    FROM amazon_products
    GROUP BY review_id
    ORDER BY 2 DESC
    LIMIT 10
    ;
""")

In [16]:
most_total_votes.show()

+--------------+--------------------------------+
|     review_id|sum(CAST(total_votes AS DOUBLE))|
+--------------+--------------------------------+
|R3R24QH3CDS83N|                         48362.0|
|R2XKMDXZHQ26YX|                         41889.0|
|R19XO9PS38WRWO|                         41866.0|
|R2YVZNKUMWGYJ4|                         32373.0|
|R1PL1U31XQG8KG|                         32166.0|
| RBUNL9QWDYTD7|                         29433.0|
|R2JF7E91JJVHAT|                         28727.0|
|R3PG4OX6C5KVN4|                         26382.0|
| RV0R3AODMRNJZ|                         26143.0|
| RXXPVOUH9NLL3|                         24170.0|
+--------------+--------------------------------+



In [39]:
most_helpful_reviews = spark.sql("""
    SELECT review_id, LENGTH(review_headline), LENGTH(review_body), sum(helpful_votes), sum(total_votes)
    FROM amazon_products
    GROUP BY review_id, review_headline, review_body
    ORDER BY 4 DESC
    LIMIT 10
    ;
""")

In [ ]:
most_helpful_reviews.show()

+--------------+-----------------------+-------------------+----------------------------------+
|     review_id|length(review_headline)|length(review_body)|sum(CAST(helpful_votes AS DOUBLE))|
+--------------+-----------------------+-------------------+----------------------------------+
|R3R24QH3CDS83N|                     41|              12906|                           47524.0|
|R19XO9PS38WRWO|                      8|                863|                           41393.0|
|R2XKMDXZHQ26YX|                     20|               1566|                           41278.0|
|R2YVZNKUMWGYJ4|                     25|              23069|                           31924.0|
|R1PL1U31XQG8KG|                     24|              13594|                           31417.0|
| RBUNL9QWDYTD7|                     69|              29778|                           28611.0|
|R2JF7E91JJVHAT|                     28|               2849|                           27550.0|
|R3PG4OX6C5KVN4|                     33|

In [46]:
least_helpful_reviews = spark.sql("""
    SELECT * FROM
    (
    SELECT review_id, LENGTH(review_headline) AS LRH, LENGTH(review_body) AS LRB, sum(helpful_votes) AS SMV
    FROM amazon_products
    GROUP BY review_id, review_headline, review_body) AS T1
    WHERE SMV IS NOT NULL
    ORDER BY 4 ASC
    LIMIT 10
    ;
""")

In [47]:
least_helpful_reviews.show()

+--------------+---+---+---+
|     review_id|LRH|LRB|SMV|
+--------------+---+---+---+
|R3C5K1E2HEM382| 40|133|0.0|
|R36TNVTLPVW9RG| 12| 80|0.0|
|R249VPTSYSA70J| 13|309|0.0|
|R338GMJOG6092P| 10| 11|0.0|
| R3ZUOTGPISO7C| 38|162|0.0|
|R2VC99EDMJAY18| 65|113|0.0|
| RVLME3SA3S153| 41| 45|0.0|
|R1G163R7K5VEEI|127|647|0.0|
|R25LMZRNUT0DCL| 11| 34|0.0|
| R7V4O1DERK0JT| 42|129|0.0|
+--------------+---+---+---+



In [48]:
#Do you see any correlation between how many reviews a certain customer (customer_id) published and the "helpfulness" of the reviews?
helpfulness_and_review_count = spark.sql("""
    SELECT customer_id, sum(helpful_votes), count(review_id)
    FROM amazon_products
    GROUP BY customer_id
    order by 2 DESC
    LIMIT 10
    ;
""")

In [49]:
helpfulness_and_review_count.show()

+-----------+----------------------------------+----------------+
|customer_id|sum(CAST(helpful_votes AS DOUBLE))|count(review_id)|
+-----------+----------------------------------+----------------+
|   50122160|                          259768.0|           59623|
|    9286343|                          143066.0|              22|
|   50732546|                          121866.0|           30761|
|   44834233|                          114651.0|            4046|
|   52496677|                           93644.0|           14391|
|   52615377|                           87930.0|            4264|
|   50881246|                           87347.0|            7403|
|   52500542|                           79976.0|            2824|
|   50913245|                           76918.0|            6301|
|   30669680|                           75286.0|             722|
+-----------+----------------------------------+----------------+



In [51]:
review_count_and_helpfulness = spark.sql("""
    SELECT customer_id, sum(helpful_votes), count(review_id)
    FROM amazon_products
    GROUP BY customer_id
    order by 3 DESC
    LIMIT 10
    ;
""")

In [52]:
review_count_and_helpfulness.show()

+-----------+----------------------------------+----------------+
|customer_id|sum(CAST(helpful_votes AS DOUBLE))|count(review_id)|
+-----------+----------------------------------+----------------+
|   50122160|                          259768.0|           59623|
|   50732546|                          121866.0|           30761|
|   52496677|                           93644.0|           14391|
|   52004451|                           17605.0|           10905|
|   50736950|                           20222.0|           10129|
|   18116317|                           27862.0|            8727|
|   44731853|                           16578.0|            7593|
|   50881246|                           87347.0|            7403|
|   38002140|                            9446.0|            7197|
|   36642996|                           46232.0|            6988|
+-----------+----------------------------------+----------------+



In [ ]:
#R3R24QH3CDS83N review id gets the most total votes with 48362.
#R3R24QH3CDS83N review id gets the most helpful votes with 47,524 points.
#R3C5K1E2HEM382 along with a few more review id gets the least helpful votes with 0 points.
#There doesn't appear to be a significant correlation between the length of the review headline or review body and the 'helpfulness' of a review. For instance, the second most 'helpful' review, identified as R19XO9PS38WRWO, has a concise review headline of just 8 characters, coupled with an extensive review body spanning 863 characters. Conversely, one of the least 'helpful' reviews, R1G163R7K5VEEI, possesses a similarly lengthy review body, with 647 characters. Likewise, another review that is considered one of the least helpful, R338GMJOG6092P, features a review headline of only 10 characters.
#There doesn't appear to be a significant correlation between how many reviews a certain customer has published and the "helpfulness" of the reviews. The data bolster this fact, for instance, the customer id, 9286343, with the second most helpful votes (around 143,066) has only published 22 reviews to date. On the other hand, there is a customer, 38002140, who has published 7197 reviews, but only has 9,446 helpful votes.
#That being said, it is worth pointing out that the customer, 50122160, have the most reviews with the most "helpful" votes!